# Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import csv
import nltk
import string
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from matplotlib import pyplot as plt
from sklearn import datasets, linear_model
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV
import joblib
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import TweetTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.classify.scikitlearn import SklearnClassifier
from textblob import TextBlob
from bs4 import BeautifulSoup

import pickle
import re, string
from collections import Counter
from string import punctuation
import tweepy
from tweepy import OAuthHandler
import json
from wordcloud import WordCloud

import plotly
import chart_studio.plotly as py
import plotly.graph_objs as go 
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

from IPython.display import IFrame
import folium
from folium import plugins
from folium.plugins import MarkerCluster, FastMarkerCluster, HeatMapWithTime


pd.set_option('display.max_colwidth', 1)
plt.style.use('seaborn-white')

# Process the train data

In [2]:
#label 0-negative, 2-means positive, 4-neutral
data = pd.read_csv("trainingdata_english.csv",skip_blank_lines=True,encoding = "latin") 
data

,label,tweet
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there."
...,...,...
1212542,4,dont ignore me
1212543,4,I'm comin wit u!!
1212544,4,"I want to go to VP, but no one is willing to come with me."
1212545,4,"Wah, why are you sad?"


In [4]:
# check the number of positive vs. negative tagged sentences
negatives = data['label'][data.label == 0]
positives = data['label'][data.label == 2]
neutral = data['label'][data.label == 4]

print('number of negative tagged sentences is: {}'.format(len(negatives)))
print('number of positive tagged sentences is: {}'.format(len(positives)))
print('number of neutral tagged sentences  is: {}'.format(len(neutral)))
print('total length of the data is:            {}'.format(data.shape[0]))

number of negative tagged sentences is: 600000
number of positive tagged sentences is: 600000
number of neutral tagged sentences  is: 12547
total length of the data is:            1212547


# Natural Language Processing (NLP)

###### Preprocessing 1:  Clean tweet text by removing links, special characters

In [5]:
def text_cleaning(tweet):
    tweet = tweet.lower() # convert text to lower-case
    tweet = re.sub(r'\&\w*;', '',tweet) # remove HTML special entities (e.g. &amp;)
    tweet = re.sub('((www\.[^\s]+)|(https?://[^\s]+))', 'URL', tweet) # remove URLs
    tweet = re.sub('@[^\s]+', 'AT_USER', tweet) # remove usernames
    tweet = re.sub(r'\$\w*', '', tweet) # remove tickers
    tweet = re.sub(r'#\w*', '', tweet) # remove hashtags
    tweet = re.sub(r'https?:\/\/.*\/\w*', '', tweet) # remove hyperlinks
    tweet = tweet.lstrip(' ') # remove single space remaining at the front of the tweet.
    tweet = re.sub(r'\s\s+', ' ', tweet) # remove whitespace (including new line characters) 
    tweet = re.sub(r'[' + punctuation.replace('@', '') + ']+', ' ', tweet) # remove punctuation and split 's, 't, 've with a space for filter
    tweet = re.sub(r'\b\w{1,2}\b', '', tweet) # remove words with 2 or fewer letters 
    tweet = tweet.lstrip(' ') # remove single space remaining at the front of the tweet.
  
    # remove characters beyond Basic Multilingual Plane (BMP) of unicode (contains characters for almost all modern languages, and a large number of symbols):
    tweet = ''.join(c for c in tweet if c <= '\uFFFF') 
    return tweet

    # clean dataframe's text column
    data['tweet'] = data['tweet'].apply(text_cleaning)
    # preview some cleaned tweets
    data['tweet'].head()

In [6]:
print(data.dtypes)

label    int64 
tweet    object
dtype: object


In [7]:
before = data['tweet'][600]
after = text_cleaning(before)

before,'\n',after

('@Born_4_Broadway Lost  and it was St. Ignacius Prepatory School. Haha.',
 '\n',
 'USER lost and  was   ignacius prepatory school  haha ')

In [8]:
# drop duplicates
data = data.drop_duplicates('tweet')
data.shape

(1198985, 2)

###### Pre-processing 2: Tokenize without out Stop-Words

In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nanan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
# show stop words examples
stop_words = stopwords.words('english') # show some stop words
' , '.join(stop_words)[:200], len(stop_words)

("i , me , my , myself , we , our , ours , ourselves , you , you're , you've , you'll , you'd , your , yours , yourself , yourselves , he , him , his , himself , she , she's , her , hers , herself , it ",
 179)

In [11]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nanan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nanan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
# tokenize helper function
def process_text(text):
    nopunc = [char for char in list(text) if char not in string.punctuation] # check characters to see if they are in punctuation
    nopunc = ''.join(nopunc) # join the characters again to form the string
    return [word for word in nopunc.lower().split() if word.lower() not in stopwords.words('english')] # remove any stopwords

def remove_words(word_list):
    remove = ['com','pic','twitter','...','“','”','’','…']
    return [w for w in word_list if w not in remove]

def lemmatize_tokenize(text):
    lemmatizer = WordNetLemmatizer()
    return [lemmatizer.lemmatize(token) for token in word_tokenize(text)]

# tokenize message column and create a column for tokens
data = data.copy()
data['tokens'] = data['tweet'].apply(process_text)
data['words'] = data['tokens'].apply(remove_words)
data['lemmatize'] = data['tweet'].apply(lemmatize_tokenize)
data.head()

,label,tweet,tokens,words
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D","[switchfoot, httptwitpiccom2y1zl, awww, thats, bummer, shoulda, got, david, carr, third, day]","[switchfoot, httptwitpiccom2y1zl, awww, thats, bummer, shoulda, got, david, carr, third, day]"
1,0,is upset that he can't update his Facebook by texting it... and might cry as a result School today also. Blah!,"[upset, cant, update, facebook, texting, might, cry, result, school, today, also, blah]","[upset, cant, update, facebook, texting, might, cry, result, school, today, also, blah]"
2,0,@Kenichan I dived many times for the ball. Managed to save 50% The rest go out of bounds,"[kenichan, dived, many, times, ball, managed, save, 50, rest, go, bounds]","[kenichan, dived, many, times, ball, managed, save, 50, rest, go, bounds]"
3,0,my whole body feels itchy and like its on fire,"[whole, body, feels, itchy, like, fire]","[whole, body, feels, itchy, like, fire]"
4,0,"@nationwideclass no, it's not behaving at all. i'm mad. why am i here? because I can't see you all over there.","[nationwideclass, behaving, im, mad, cant, see]","[nationwideclass, behaving, im, mad, cant, see]"


In [13]:
# style 1 tokens
data['tokens'].head(10)

0    [switchfoot, httptwitpiccom2y1zl, awww, thats, bummer, shoulda, got, david, carr, third, day]
1    [upset, cant, update, facebook, texting, might, cry, result, school, today, also, blah]      
2    [kenichan, dived, many, times, ball, managed, save, 50, rest, go, bounds]                    
3    [whole, body, feels, itchy, like, fire]                                                      
4    [nationwideclass, behaving, im, mad, cant, see]                                              
5    [kwesidei, whole, crew]                                                                      
6    [need, hug]                                                                                  
7    [loltrish, hey, long, time, see, yes, rains, bit, bit, lol, im, fine, thanks, hows]          
8    [tatianak, nope, didnt]                                                                      
9    [twittera, que, muera]                                                                       
Name: toke

###### Pre-processing 3: Feature Extraction

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
# vectorize (convert each message which is represented by a list of tokens into a vector that a machine learning model can understand)
# converts a collection of text documents to a matrix of token counts
bow_transformer = CountVectorizer(analyzer=process_text).fit(data['tweet']) 
# print total number of vocab words
print(len(bow_transformer.vocabulary_))

686169


In [15]:
bow_transformer.vocabulary_

{'switchfoot': 586774,
 'httptwitpiccom2y1zl': 275231,
 'awww': 53262,
 'thats': 598044,
 'bummer': 88902,
 'shoulda': 550162,
 'got': 217465,
 'david': 138818,
 'carr': 97598,
 'third': 604607,
 'day': 139703,
 'upset': 634437,
 'cant': 95197,
 'update': 633892,
 'facebook': 183836,
 'texting': 596539,
 'might': 407154,
 'cry': 129356,
 'result': 515240,
 'school': 536560,
 'today': 611884,
 'also': 30732,
 'blah': 73151,
 'kenichan': 344071,
 'dived': 152443,
 'many': 390250,
 'times': 608794,
 'ball': 57514,
 'managed': 388838,
 'save': 534662,
 '50': 9889,
 'rest': 515003,
 'go': 214361,
 'bounds': 80142,
 'whole': 654391,
 'body': 76901,
 'feels': 188484,
 'itchy': 309385,
 'like': 368375,
 'fire': 191807,
 'nationwideclass': 434002,
 'behaving': 64632,
 'im': 300985,
 'mad': 385133,
 'see': 540001,
 'kwesidei': 355371,
 'crew': 127957,
 'need': 435435,
 'hug': 293413,
 'loltrish': 375452,
 'hey': 236184,
 'long': 375888,
 'time': 608467,
 'yes': 672532,
 'rains': 505304,
 'bit': 

In [16]:
# example of vectorized text
sample_tweet = data['tweet'][111]
print(sample_tweet)
print('\n')
# vector representation
bow_sample = bow_transformer.transform([sample_tweet])
print(bow_sample)
print('\n')

I'm sooo sad!!! they killed off Kutner on House  whyyyyyyyy


  (0, 244194)	1
  (0, 300985)	1
  (0, 347177)	1
  (0, 355165)	1
  (0, 528526)	1
  (0, 564621)	1
  (0, 654972)	1




In [17]:
# transform the entire DataFrame of messages
messages_bow = bow_transformer.transform(data['tweet'])

# check out the bag-of-words counts for the entire corpus as a large sparse matrix
print('Shape of Sparse Matrix: ', messages_bow.shape)
print('Amount of Non-Zero occurences: ', messages_bow.nnz)

Shape of Sparse Matrix:  (1198985, 686169)
Amount of Non-Zero occurences:  9201789


###### Term Frequency, Inverse Document Frequency

In [18]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(messages_bow)
tfidf_sample = tfidf_transformer.transform(bow_sample)
print(tfidf_sample)

  (0, 654972)	0.5728549514754353
  (0, 564621)	0.30086918392013245
  (0, 528526)	0.2319661525544924
  (0, 355165)	0.5430683528105762
  (0, 347177)	0.3717059598397657
  (0, 300985)	0.14977070356375646
  (0, 244194)	0.268303601831209


In [19]:
# to transform the entire bag-of-words corpus
tweet_tfidf = tfidf_transformer.transform(messages_bow)
print(tweet_tfidf.shape)

(1198985, 686169)


## Model Training and Evaluation

#### Random Forest Classifier

In [16]:
# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', RandomForestClassifier()),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__n_estimators': [20, 50],
                'clf__max_features': ['auto'],
                'clf__max_depth' : [4,5,6],
                'clf__criterion' :['gini', 'entropy'],
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.7min
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed: 40.9min
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed: 125.2min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed: 138.7min finished



Best Model: 0.533935 using {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 6, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': False}


Mean: 0.524345 Stdev:(0.005149) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': True}
Mean: 0.526015 Stdev:(0.008175) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 20, 'tfidf__use_idf': False}
Mean: 0.519057 Stdev:(0.000766) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 50, 'tfidf__use_idf': True}
Mean: 0.518440 Stdev:(0.000366) with: {'bow__ngram_range': (1, 1), 'clf__criterion': 'gini', 'clf__max_depth': 4, 'clf__max_features': 'auto', 'clf__n_estimators': 50, 'tfidf__use_idf': False}
Mean: 0.528067 Stdev:(0.008229) with: {'bow__

#### Naive Bayes

In [19]:
# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)


# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', MultinomialNB()),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__alpha': (1e-2, 1e-3),
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 8 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 21.6min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 65.7min finished



Best Model: 0.836509 using {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': False}


Mean: 0.803009 Stdev:(0.001064) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.818192 Stdev:(0.001022) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.799113 Stdev:(0.000957) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.812924 Stdev:(0.001147) with: {'bow__ngram_range': (1, 1), 'clf__alpha': 0.001, 'tfidf__use_idf': False}
Mean: 0.823180 Stdev:(0.001459) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': True}
Mean: 0.836509 Stdev:(0.001448) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.01, 'tfidf__use_idf': False}
Mean: 0.810987 Stdev:(0.001503) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.001, 'tfidf__use_idf': True}
Mean: 0.825470 Stdev:(0.001383) with: {'bow__ngram_range': (1, 2), 'clf__alpha': 0.001, 'tfidf__use_idf': False}


In [21]:
# save best model to current working directory
joblib.dump(grid, "twitter_sentiment_NB.pkl")

['twitter_sentiment_NB.pkl']

#### SVM Classifier

In [ ]:
from imblearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier 

# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', SGDClassifier(class_weight='balanced')),  
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__loss' : ["log", "squared_hinge", "modified_huber"],
                'clf__alpha': (1e-3, 1e-4),
                'clf__penalty' : ["l2", "none"],
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, n_jobs=-1, verbose=1)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  5.6min


In [20]:
from imblearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

# Run Train Data Through Pipeline analyzer=text_process
X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['label'], test_size=0.2)

# create pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(strip_accents='ascii',
                            lowercase=True)),  
    ('tfidf', TfidfTransformer()),  
    ('clf', OneVsRestClassifier(LinearSVC(class_weight='balanced', max_iter=100000))) 
])

# this is where we define the values for GridSearchCV to iterate over
parameters = [
            {
                'bow__ngram_range': [(1,1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__estimator__loss' : ["hinge", "squared_hinge"],
                'clf__estimator__tol' : [1e-4, 1e-5],
                'clf__estimator__penalty' : ["l2"]
            },
            ]

# do 10-fold cross validation for each of the 6 possible combinations of the above params
grid = GridSearchCV(pipeline, cv=10, param_grid=parameters, verbose=1, n_jobs= 8)
grid.fit(X_train,y_train)

# summarize results
print("\nBest Model: %f using %s" % (grid.best_score_, grid.best_params_))
print('\n')
means = grid.cv_results_['mean_test_score']
stds = grid.cv_results_['std_test_score']
params = grid.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("Mean: %f Stdev:(%f) with: %r" % (mean, stdev, param))

Fitting 10 folds for each of 16 candidates, totalling 160 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed: 113.2min
[Parallel(n_jobs=8)]: Done 160 out of 160 | elapsed: 435.8min finished



Best Model: 0.812328 using {'bow__ngram_range': (1, 2), 'clf__estimator__loss': 'hinge', 'clf__estimator__penalty': 'l2', 'clf__estimator__tol': 0.0001, 'tfidf__use_idf': True}


Mean: 0.752537 Stdev:(0.001417) with: {'bow__ngram_range': (1, 1), 'clf__estimator__loss': 'hinge', 'clf__estimator__penalty': 'l2', 'clf__estimator__tol': 0.0001, 'tfidf__use_idf': True}
Mean: 0.750133 Stdev:(0.001144) with: {'bow__ngram_range': (1, 1), 'clf__estimator__loss': 'hinge', 'clf__estimator__penalty': 'l2', 'clf__estimator__tol': 0.0001, 'tfidf__use_idf': False}
Mean: 0.752534 Stdev:(0.001417) with: {'bow__ngram_range': (1, 1), 'clf__estimator__loss': 'hinge', 'clf__estimator__penalty': 'l2', 'clf__estimator__tol': 1e-05, 'tfidf__use_idf': True}
Mean: 0.750134 Stdev:(0.001145) with: {'bow__ngram_range': (1, 1), 'clf__estimator__loss': 'hinge', 'clf__estimator__penalty': 'l2', 'clf__estimator__tol': 1e-05, 'tfidf__use_idf': False}
Mean: 0.746221 Stdev:(0.001099) with: {'bow__ngram_range': (1, 1), '

In [21]:
# save best model to current working directory
joblib.dump(grid, "twitter_sentiment_LSVM_english.pkl")

['twitter_sentiment_LSVM_english.pkl']

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.metrics import auc,roc_curve

In [24]:
model_SVM = joblib.load("twitter_sentiment_LSVM_english.pkl" )

# get predictions from best model above
y_preds = model_SVM.predict(X_test)

print('accuracy score: ',accuracy_score(y_test, y_preds))
print('\n')
print('confusion matrix: \n',confusion_matrix(y_test,y_preds))
print('\n')
print(metrics.classification_report(y_test, y_preds))

accuracy score:  0.8110693628360655


confusion matrix: 
 [[97379 19955   963]
 [21659 97073   399]
 [ 1217  1112    40]]


              precision    recall  f1-score   support

           0       0.81      0.82      0.82    118297
           2       0.82      0.81      0.82    119131
           4       0.03      0.02      0.02      2369

    accuracy                           0.81    239797
   macro avg       0.55      0.55      0.55    239797
weighted avg       0.81      0.81      0.81    239797

